In [1]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from keras.layers import Input, Convolution1D, MaxPooling1D, Dense, Flatten, Reshape, UpSampling1D, Dropout
from keras.models import Layer, Model

from keras.optimizers import SGD

import keras.backend as K
import numpy as np
seed=6
np.random.seed(seed)

Using Theano backend.


In [2]:
aa_length=10
filter_length=3
nb_filter=5

In [3]:
conv = Convolution1D(nb_filter, filter_length,
                               init='glorot_normal',
                               activation='relu',
                               border_mode='same',
                               name='Conv1')

In [4]:
from keras import activations, initializations, regularizers, constraints
from keras.engine import InputSpec
from keras.utils.np_utils import conv_output_length


class Deconvolution1D(Layer):

    def __init__(self, bound_conv_layer,
                 init='uniform', activation='linear', weights=None, subsample_length=1,
                 W_regularizer=None, b_regularizer=None, activity_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, input_dim=None, input_length=None, **kwargs):

        if 'border_mode' in kwargs:
            raise Exception('Border mode is infered from Conv Layer')

        self._bound_conv_layer = bound_conv_layer
        try:
            self.nb_filter = self._bound_conv_layer.input_shape[2]
        except Exception:
            self.nb_filter = 'Not sure yet, input shape of convolutional layer not provided during construction.'
        self.filter_length = self._bound_conv_layer.filter_length
        self.border_mode = self._bound_conv_layer.border_mode

        self.init = initializations.get(init, dim_ordering='th')
        self.activation = activations.get(activation)
        self.subsample_length = subsample_length

        self.subsample = (subsample_length, 1)

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.input_spec = [InputSpec(ndim=3)]
        self.initial_weights = weights
        self.input_dim = input_dim
        self.input_length = input_length
        if self.input_dim:
            kwargs['input_shape'] = (self.input_length, self.input_dim)
        super(Deconvolution1D, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[2]
        self.nb_filter = self._bound_conv_layer.input_shape[2]
        self.W_shape = (self.filter_length, 1, self.input_dim, self.nb_filter)
        self.W = self._bound_conv_layer.W.dimshuffle((0, 1, 3, 2))
        if self.bias:
            self.b = K.zeros((self.nb_filter,), name='{}_b'.format(self.name))
            self.trainable_weights = [self.b]
        else:
            self.trainable_weights = []
        self.regularizers = []

        if self.W_regularizer:
            self.W_regularizer.set_param(self.W)
            self.regularizers.append(self.W_regularizer)

        if self.bias and self.b_regularizer:
            self.b_regularizer.set_param(self.b)
            self.regularizers.append(self.b_regularizer)

        if self.activity_regularizer:
            self.activity_regularizer.set_layer(self)
            self.regularizers.append(self.activity_regularizer)

        self.constraints = {}
        if self.W_constraint:
            self.constraints[self.W] = self.W_constraint
        if self.bias and self.b_constraint:
            self.constraints[self.b] = self.b_constraint

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def get_output_shape_for(self, input_shape):
        length = conv_output_length(input_shape[1],
                                    self.filter_length,
                                    self.border_mode,
                                    self.subsample[0])
        return (input_shape[0], length, self.nb_filter)

    def call(self, x, mask=None):
        x = K.expand_dims(x, 2)  # add a dummy dimension
        output = K.conv2d(x, self.W, strides=self.subsample,
                          border_mode=self.border_mode,
                          dim_ordering='tf')
        output = K.squeeze(output, 2)  # remove the dummy dimension
        if self.bias:
            output += K.reshape(self.b, (1, 1, self.nb_filter))
        output = self.activation(output)
        return output

    def get_config(self):
        config = {'nb_filter': self.nb_filter,
                  'filter_length': self.filter_length,
                  'init': self.init.__name__,
                  'activation': self.activation.__name__,
                  'border_mode': self.border_mode,
                  'subsample_length': self.subsample_length,
                  'W_regularizer': self.W_regularizer.get_config() if self.W_regularizer else None,
                  'b_regularizer': self.b_regularizer.get_config() if self.b_regularizer else None,
                  'activity_regularizer': self.activity_regularizer.get_config() if self.activity_regularizer else None,
                  'W_constraint': self.W_constraint.get_config() if self.W_constraint else None,
                  'b_constraint': self.b_constraint.get_config() if self.b_constraint else None,
                  'bias': self.bias,
                  'input_dim': self.input_dim,
                  'input_length': self.input_length}
        base_config = super(Deconvolution1D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
deconv = Deconvolution1D(conv)

In [6]:
inp = Input(shape=(aa_length, 1), name='aa_seq')
x_data = np.random.random((10, 10, 1))

In [7]:
x_c = conv(inp)
x_d = deconv(x_c)

In [8]:
coder = Model(input=inp, output=x_d)

In [9]:
coder.count_params()

21

In [10]:
coder.compile(optimizer=SGD(lr=0.1,
                            decay=1e-6, momentum=0.9, nesterov=True), loss='mse')

In [11]:
coder.fit(x_data, x_data, nb_epoch=100)

Epoch 1/100
10/10 [==============================] - 0s - loss: 0.2153
Epoch 2/100
10/10 [==============================] - 0s - loss: 0.1129
Epoch 3/100
10/10 [==============================] - 0s - loss: 0.0982
Epoch 4/100
10/10 [==============================] - 0s - loss: 0.0685
Epoch 5/100
10/10 [==============================] - 0s - loss: 0.0578
Epoch 6/100
10/10 [==============================] - 0s - loss: 0.0523
Epoch 7/100
10/10 [==============================] - 0s - loss: 0.0484
Epoch 8/100
10/10 [==============================] - 0s - loss: 0.0455
Epoch 9/100
10/10 [==============================] - 0s - loss: 0.0429
Epoch 10/100
10/10 [==============================] - 0s - loss: 0.0403
Epoch 11/100
10/10 [==============================] - 0s - loss: 0.0377
Epoch 12/100
10/10 [==============================] - 0s - loss: 0.0350
Epoch 13/100
10/10 [==============================] - 0s - loss: 0.0323
Epoch 14/100
10/10 [==============================] - 0s - loss: 0.0295
E

In [12]:
print(x_data.squeeze()[0])
print(coder.predict(x_data).squeeze()[0])
print(coder.evaluate(x_data, x_data))

[ 0.89286015  0.33197981  0.82122912  0.04169663  0.10765668  0.59505206
  0.52981736  0.41880743  0.33540785  0.62251943]
[ 0.86979097  0.32658657  0.83307517  0.06249763  0.11308175  0.60168821
  0.53057754  0.42141896  0.33680576  0.61647892]
10/10 [==============================] - 0s
8.31144789117e-05


In [13]:
np.mean(np.square(x_data-coder.predict(x_data)), dtype=np.float32)

8.3114486e-05